In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
import local_python_packages.features_adding as local

In [2]:
pd.set_option('display.max_columns', None)
flights_data = pd.read_csv('UA flights 2019.csv',parse_dates=[0])
flights_data = flights_data.sort_values(['fl_date'])

In [ ]:
df_weather = pd.read_csv('cities_and_dates_weather_final.csv',parse_dates=[1])

In [3]:
df_holidays = pd.read_csv('US holidays.csv',parse_dates=[0])

Adding additional columns from our feature engineering

In [ ]:
flights_data = local.add_taxi_Ndays_rolling(flights_data, 30)

In [ ]:
flights_data = local.add_traffic_rolling(flights_data, 30)

In [ ]:
flights_data = local.make_month_dummies(flights_data, 'fl_date')

In [ ]:
flights_data = local.merging_weather_flights(flights_data,df_weather)

In [ ]:
flights_data = local.add_dep_delay_Ndays_rolling(flights_data, 30)

In [4]:
flights_data = local.add_US_holidays(flights_data, df_holidays)

In [ ]:
flights_data = local.make_dates_ordinal(flights_data, 'fl_date')

KeyError: 'dest_visibility'

In [ ]:
features_list = ['fl_date','taxi_out','taxi_in', 'arr_delay',
                 'crs_elapsed_time',
       'air_time', 'distance',
       '30d taxi_out', '30d taxi_in',
       '30d roll flts origin_airport_id', '30d roll flts dest_airport_id',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'origin_city_wspd', 'origin_visibility',
       'dest_city_wspd', 'dest_visibility', 'origin_cond_Overcast',
       'origin_cond_Partially cloudy', 'origin_cond_Rain', 'origin_cond_Snow',
       'dest_cond_Overcast', 'dest_cond_Partially cloudy', 'dest_cond_Rain',
       'dest_cond_Snow','30 days roll dep_time']

In [ ]:
flights_data = local.distill_features(flights_data,desired_features=features_list)

In [ ]:
flights_data.shape

In [ ]:
flights_data = local.replace_nan_with_mean(flights_data, 'arr_delay')

In [ ]:
features_list.remove('arr_delay')

In [ ]:
flights_data.shape

In [ ]:
flights_data = flights_data.dropna()

In [ ]:
flights_data.shape

In [ ]:
X = flights_data[features_list]
y = flights_data['arr_delay']

In [ ]:
scaler = StandardScaler()

In [ ]:
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = local.quick_split(X,y,train_ratio=0.75)

In [ ]:
sgd = SGDRegressor(max_iter = 10000)

In [ ]:
parameters = {'penalty': ['l2', 'l1', 'elasticnet'],
             'alpha':[0.0001, 0.001, 0.01],
             'loss':['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive']}

In [ ]:
GrdSrch = GridSearchCV(sgd, param_grid=parameters, scoring=['neg_mean_absolute_error', 'r2'],
                       refit='r2', n_jobs = 5, verbose= 4)

In [ ]:
GrdSrch.fit(X_train, y_train)

In [ ]:
y_pred=GrdSrch.best_estimator_.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
GrdSrch.best_score_

In [ ]:
GrdSrch.best_params_

In [ ]:
GrdSrch.best_estimator_.coef_

In [ ]:
features_list[4]